In [1]:
import os
import pandas as pd
from glob import glob
# import textstat
from data_utils import (
    clean_transcripts,
    flesch,
    fog,
    smog,
    terms,
    uniqueterms, 
    ttr, 
    mtld, 
    hdd
)

# FP_CNN = '../../readable_news_lf/cnn_cc/'
FP_CNN = 'raw_data/cnn_cc/'
files = glob(os.path.join(FP_CNN, '*.zip'))
# files.append('../../readable_news_lf/cnn_cc/cnn-7.csv.gz')
files.append('raw_data/cnn_cc/cnn-7.csv.gz')
files

['raw_data/cnn_cc/cnn-6.zip',
 'raw_data/cnn_cc/cnn-1.zip',
 'raw_data/cnn_cc/cnn-3.zip',
 'raw_data/cnn_cc/cnn-4.zip',
 'raw_data/cnn_cc/cnn-2.zip',
 'raw_data/cnn_cc/cnn-5.zip',
 'raw_data/cnn_cc/cnn-7.csv.gz']

In [6]:
%%time
_usecols = ['program.name', 'year', 'month', 'date', 'time', 'timezone', 'subhead', 'text']
df = (pd.concat([pd.read_csv(file, usecols=_usecols) for file in files])
      .query('text==text')
      .reset_index(drop=True)
      .assign(
          text=lambda df: df.text.apply(clean_transcripts),
          # Compute readability for text
          flesch_reading_ease=lambda df: df.text.apply(flesch),
          fog=lambda df: df.text.apply(fog), 
          smog=lambda df: df.text.apply(smog),  
          terms=lambda df: df.text.apply(terms),
          # Compute lexical richness for text
          uniqueterms=lambda df: df.text.apply(uniqueterms), 
          ttr=lambda df: df.text.apply(ttr),
          mtld=lambda df: df.text.apply(mtld),
          hdd=lambda df: df.text.apply(hdd),    
      )      
     )

CPU times: user 2d 21h 10min 16s, sys: 2min 31s, total: 2d 21h 12min 47s
Wall time: 2d 21h 13min 15s


In [7]:
df.head(5)

,program.name,year,month,date,time,timezone,subhead,text,flesch_reading_ease,fog,smog,terms,uniqueterms,ttr,mtld,hdd
0,CNN LIVE EVENT/SPECIAL,2012.0,5.0,19.0,14:30,ET,The Fighting Prince,"then and now. at his happiest, in combat unifo...",77.515372,8.344100,8.250249,3459,982,0.272854,82.194127,0.861968
1,CNN NEWSROOM,2012.0,5.0,19.0,17:00,ET,"G-8 Talks Cover Sanctions, Possible Oil Disrup...","it's the economy, stupid, and not just in the ...",74.679795,8.841846,8.855280,6152,1536,0.242616,83.549220,0.877927
2,CNN NEWSROOM,2012.0,5.0,19.0,15:00,ET,Wildfires Intensify in Arizona; Storm Bears Do...,"hello, everyone. i'm fredricka whitfield. you'...",73.594620,8.447377,9.583585,6345,1530,0.237246,86.857237,0.877840
3,CNN NEWSROOM,2012.0,5.0,19.0,16:00,ET,Terrorism Threats in Chicago; Chinese Activist...,"all right, welcome back to the newsroom. i'm f...",69.167932,10.722255,10.436528,3252,1013,0.304570,82.217656,0.868728
4,CNN NEWSROOM,2012.0,5.0,19.0,19:00,ET,Summit Cities Brace for Trouble; World Economy...,"hello, everyone. i'm don lemon. it's the top o...",69.747716,11.602472,10.249300,6931,1648,0.230619,86.933667,0.885053


In [10]:
(df
 .drop(['text'], axis=1)
 .to_csv('data/cnn_2000_2021_textstat.csv.gz', compression="gzip")
)